Analysing potential factors that contribute to access to healthy food in 2015.

In [2]:
## Dependencies
from matplotlib import pyplot as plt
from scipy import stats
import numpy as np
import pandas as pd



In [3]:

# Import the Excel files and read the tabs we need into dataframes.

#Food Environment Atlas -  DataDownload
xls1 = pd.ExcelFile('DataDownload.xls')

##Access Dataframe 
df1 = pd.read_excel(xls1, 'ACCESS')
df1 = df1[["FIPS", "State", "County","LACCESS_POP15","PCT_LACCESS_POP15"]]
df1= df1.rename(columns = {'LACCESS_POP15':'Population with low access',
                           'PCT_LACCESS_POP15':'Percent of population with low access'})

##Income Dataframe
df2 = pd.read_excel(xls1,'SOCIOECONOMIC')
df2 = df2[["FIPS", "State", "County","MEDHHINC15"]]
df2 = df2.rename(columns = {"MEDHHINC15" : "Median Household Income 2015"})





#County Level Data - Education
xls2 = pd.ExcelFile('Education.xls')

##Education Dataframe
df3 = pd.read_excel(xls2,skiprows=4)

##Trim Education Dataframe to include only the columns we need from 2013-17

df3=df3[["FIPS Code",
      "Less than a high school diploma, 2013-17",
      "High school diploma only, 2013-17",
      "Some college or associate's degree, 2013-17",
      "Bachelor's degree or higher, 2013-17",
      "Percent of adults with less than a high school diploma, 2013-17",
      "Percent of adults with a high school diploma only, 2013-17",
      "Percent of adults completing some college or associate's degree, 2013-17",
      "Percent of adults with a bachelor's degree or higher, 2013-17"
     ]]

df3= df3.rename(columns = {"FIPS Code":"FIPS",
                            "Percent of adults with less than a high school diploma, 2013-17":"% of adults with less than a high school diploma, 2013-17",
                            "Percent of adults with a high school diploma only, 2013-17":"% of adults with a high school diploma only, 2013-17",
                            "Percent of adults completing some college or associate's degree, 2013-17": "% of adults completing some college or associate's degree, 2013-17",
                            "Percent of adults with a bachelor's degree or higher, 2013-17":"% of adults with a bachelor's degree or higher, 2013-17"
                          })
                           
        


#County Level Data - PopulationEstimates
xls3 = pd.ExcelFile('PopulationEstimates.xls')

## Population Dataframe
df4 = pd.read_excel(xls3,'Population Estimates 2010-18',skiprows=2)
df4 = df4[['FIPS','POP_ESTIMATE_2015']]
df4= df4.rename(columns = {"POP_ESTIMATE_2015":"2015 Population Estimate"})


#df1.head()
#df2.head()
#df3.head()
#df4.head()

In [4]:
# Merge Population with Access

access_df = df4.merge(df1, on='FIPS')
access_df["% of population with low access"]= (access_df["Population with low access"]/access_df["2015 Population Estimate"])*100

# Change order of columns
access_df=access_df[["FIPS", "State", "County","2015 Population Estimate", "Population with low access","% of population with low access"]]

#access_df.head()

In [5]:
# Merge Access df with Income

income_df = access_df.merge(df2)
#income_df.head()

In [6]:
# Merge Income with Education to have all dataframes merged into one

access_data = income_df.merge(df3)

# Create csv file

access_data.to_csv('Resources/data.csv')

#access_data.head()

In [7]:
# look at data for the state of Colorado

CO_data=access_data[access_data['State'] =='CO']

# Verify there are no blank cells in this df
CO_data.count()


CO_data.head()

,FIPS,State,County,2015 Population Estimate,Population with low access,% of population with low access,Median Household Income 2015,"Less than a high school diploma, 2013-17","High school diploma only, 2013-17","Some college or associate's degree, 2013-17","Bachelor's degree or higher, 2013-17","% of adults with less than a high school diploma, 2013-17","% of adults with a high school diploma only, 2013-17","% of adults completing some college or associate's degree, 2013-17","% of adults with a bachelor's degree or higher, 2013-17"
243,8001,CO,Adams,490768,66895.714251,13.630822,62991.0,54006.0,87917.0,97054.0,71613.0,17.4,28.3,31.2,23.1
244,8003,CO,Alamosa,16314,915.186239,5.609821,40067.0,1588.0,2367.0,2987.0,2469.0,16.9,25.2,31.7,26.2
245,8005,CO,Arapahoe,630637,69481.673398,11.017697,67062.0,32522.0,87155.0,126608.0,175266.0,7.7,20.7,30.0,41.6
246,8007,CO,Archuleta,12393,3600.901552,29.055931,50361.0,902.0,2210.0,2869.0,3508.0,9.5,23.3,30.2,37.0
247,8009,CO,Baca,3553,955.224505,26.885013,35405.0,379.0,695.0,976.0,558.0,14.5,26.6,37.4,21.4


In [8]:
# Multivariate Regression
import statsmodels.api as sm

df = CO_data[["% of population with low access","2015 Population Estimate", "Median Household Income 2015", "% of adults with a bachelor's degree or higher, 2013-17"]]
df = df.rename(columns = {
    '% of population with low access' : 'access',
    '2015 Population Estimate' : 'population',
    'Median Household Income 2015' : 'income',
    "% of adults with a bachelor's degree or higher, 2013-17" : 'degree'
})

x = df[['population','income','degree']]
y = df['access']
model = sm.OLS(y, x).fit()
predictions = model.predict(x)
model.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 access   R-squared (uncentered):                   0.443
Model:                            OLS   Adj. R-squared (uncentered):              0.416
Method:                 Least Squares   F-statistic:                              16.17
Date:                Tue, 28 Jan 2020   Prob (F-statistic):                    7.58e-08
Time:                        19:33:32   Log-Likelihood:                         -296.23
No. Observations:                  64   AIC:                                      598.5
Df Residuals:                      61   BIC:                                      604.9
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
population -2.266e-05   1.94e-05     -1.166      0.248   -6.15e-05    1.62e-05
income         0.0006      0.000      3.002      0.004       0.000       0.001
degree        -0.3377      0.345     -0.979      0.331      -1.027       0.352
==============================================================================
Omnibus:                       41.876   Durbin-Watson:                   2.259
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              107.546
Skew:                           2.131   Prob(JB):                     4.43e-24
Kurtosis:                       7.709   Cond. No.                     2.11e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.11e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""